# Problem

Interpret a toy model trained to permute lists.

# Setup
(No need to read)

In [1]:
!git clone https://github.com/ckkissane/mech-interp-practice.git

Cloning into 'mech-interp-practice'...
remote: Enumerating objects: 270, done.
remote: Counting objects: 100% (89/89), done.
remote: Compressing objects: 100% (80/80), done.
remote: Total 270 (delta 44), reused 26 (delta 9), pack-reused 181
Receiving objects: 100% (270/270), 38.13 MiB | 7.28 MiB/s, done.
Resolving deltas: 100% (123/123), done.


In [2]:
# Janky code to do different setup when run in a Colab notebook vs VSCode
DEBUG_MODE = False
try:
    import google.colab
    IN_COLAB = True
    print("Running as a Colab notebook")
    %pip install git+https://github.com/neelnanda-io/TransformerLens.git
    # Install another version of node that makes PySvelte work way faster
    !curl -fsSL https://deb.nodesource.com/setup_16.x | sudo -E bash -; sudo apt-get install -y nodejs
    %pip install git+https://github.com/neelnanda-io/PySvelte.git
    # Needed for PySvelte to work, v3 came out and broke things...
    %pip install typeguard==2.13.3
except:
    IN_COLAB = False
    print("Running as a Jupyter notebook - intended for development only!")
    from IPython import get_ipython

    ipython = get_ipython()
    # Code to automatically update the HookedTransformer code as its edited without restarting the kernel
    ipython.magic("load_ext autoreload")
    ipython.magic("autoreload 2")

Running as a Colab notebook
  Cloning https://github.com/neelnanda-io/TransformerLens.git to /tmp/pip-req-build-fm35mirf
  Running command git clone --filter=blob:none --quiet https://github.com/neelnanda-io/TransformerLens.git /tmp/pip-req-build-fm35mirf
  Resolved https://github.com/neelnanda-io/TransformerLens.git to commit 54d548de4995a1ecc5b01b9c03aceaf0966c0eb3
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.7/739.7 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━

In [3]:
# Plotly needs a different renderer for VSCode/Notebooks vs Colab argh
import plotly.io as pio

if IN_COLAB or not DEBUG_MODE:
    # Thanks to annoying rendering issues, Plotly graphics will either show up in colab OR Vscode depending on the renderer - this is bad for developing demos! Thus creating a debug mode.
    pio.renderers.default = "colab"
else:
    pio.renderers.default = "png"

In [4]:
# Import stuff
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import einops
import tqdm.notebook as tqdm
import random
from pathlib import Path
import plotly.express as px
from torch.utils.data import DataLoader

from jaxtyping import Float, Int
from typing import List, Union, Optional
from functools import partial
import copy

import itertools
from transformers import AutoModelForCausalLM, AutoConfig, AutoTokenizer
import dataclasses
from dataclasses import dataclass
import datasets
from IPython.display import HTML

In [5]:
import pysvelte

import transformer_lens
import transformer_lens.utils as utils
from transformer_lens.hook_points import (
    HookedRootModule,
    HookPoint,
)  # Hooking utilities
from transformer_lens import HookedTransformer, HookedTransformerConfig, FactoredMatrix, ActivationCache

Plotting helper functions:

In [6]:
import plotly.graph_objects as go

update_layout_set = {"xaxis_range", "yaxis_range", "hovermode", "xaxis_title", "yaxis_title", "colorbar", "colorscale", "coloraxis", "title_x", "bargap", "bargroupgap", "xaxis_tickformat", "yaxis_tickformat", "title_y", "legend_title_text", "xaxis_showgrid", "xaxis_gridwidth", "xaxis_gridcolor", "yaxis_showgrid", "yaxis_gridwidth"}
def imshow(tensor, renderer=None, xaxis="", yaxis="", **kwargs):
    if isinstance(tensor, list):
        tensor = torch.stack(tensor)
    kwargs_post = {k: v for k, v in kwargs.items() if k in update_layout_set}
    kwargs_pre = {k: v for k, v in kwargs.items() if k not in update_layout_set}
    if "facet_labels" in kwargs_pre:
        facet_labels = kwargs_pre.pop("facet_labels")
    else:
        facet_labels = None
    if "color_continuous_scale" not in kwargs_pre:
        kwargs_pre["color_continuous_scale"] = "RdBu"
    fig = px.imshow(utils.to_numpy(tensor), color_continuous_midpoint=0.0,labels={"x":xaxis, "y":yaxis}, **kwargs_pre).update_layout(**kwargs_post)
    if facet_labels:
        for i, label in enumerate(facet_labels):
            fig.layout.annotations[i]['text'] = label

    fig.show(renderer)

def line(tensor, renderer=None, xaxis="", yaxis="", **kwargs):
    px.line(y=utils.to_numpy(tensor), labels={"x":xaxis, "y":yaxis}, **kwargs).show(renderer)

def scatter(x, y, xaxis="", yaxis="", caxis="", renderer=None, **kwargs):
    x = utils.to_numpy(x)
    y = utils.to_numpy(y)
    px.scatter(y=y, x=x, labels={"x":xaxis, "y":yaxis, "color":caxis}, **kwargs).show(renderer)

def lines(lines_list, x=None, mode='lines', labels=None, xaxis='', yaxis='', title = '', log_y=False, hover=None, **kwargs):
    # Helper function to plot multiple lines
    if type(lines_list)==torch.Tensor:
        lines_list = [lines_list[i] for i in range(lines_list.shape[0])]
    if x is None:
        x=np.arange(len(lines_list[0]))
    fig = go.Figure(layout={'title':title})
    fig.update_xaxes(title=xaxis)
    fig.update_yaxes(title=yaxis)
    for c, line in enumerate(lines_list):
        if type(line)==torch.Tensor:
            line = utils.to_numpy(line)
        if labels is not None:
            label = labels[c]
        else:
            label = c
        fig.add_trace(go.Scatter(x=x, y=line, mode=mode, name=label, hovertext=hover, **kwargs))
    if log_y:
        fig.update_layout(yaxis_type="log")
    fig.show()

def bar(tensor, renderer=None, xaxis="", yaxis="", **kwargs):
    px.bar(
        y=utils.to_numpy(tensor),
        labels={"x": xaxis, "y": yaxis},
        template="simple_white",
        **kwargs).show(renderer)

In [7]:
import transformer_lens.patching as patching
from transformer_lens import evals
import math

In [8]:
def visualize_attn_patterns(heads, local_tokens, local_cache, title: str = ""):
    labels = []
    patterns = []
    batch_index = 0

    for head in heads:
        if isinstance(head, tuple):
            layer, head_index = head
        else:
            layer, head_index = head // model.cfg.n_heads, head % model.cfg.n_heads
        patterns.append(local_cache["pattern", layer][batch_index, head_index])
        labels.append(f"L{layer}H{head_index}")
    patterns = torch.stack(patterns, dim=-1)
    attn_viz = pysvelte.AttentionMulti(tokens=model.to_str_tokens(local_tokens[batch_index]), attention=patterns, head_labels=labels)
    display(HTML(f"<h3>{title}</h3>"))
    attn_viz.show()

# Load Model

In [9]:
torch.set_grad_enabled(False)

In [10]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


The smallest model I found was a 2L, 1H (per layer), attn-only transformer with no layernorm or biases. Note a 1L transformer was not able to solve this task (even with up to 8 heads).

![picture](https://drive.google.com/uc?id=15DCokm0rjNEBFn5znNJ8mEu5ATF5z7CH)

The model has already been trained, and is loaded into this notebook below:

In [11]:
LIST_LEN = 3
MAX_INT = 50
cfg = HookedTransformerConfig(
    n_layers=2,
    n_heads=1,
    d_model=128,
    d_head=128,
    n_ctx=3*LIST_LEN+3, # BOS d1 d2 d3 MID p1 p2 p3 END a1 a2 a3
    d_vocab= MAX_INT+3, # 0, ..., MAX_INT-1, BOS, MID, END
    d_vocab_out=MAX_INT,
    attn_only=True,
    normalization_type=None,
    device=device,
    seed=0,
)
model = HookedTransformer(cfg)
print(model)

HookedTransformer(
  (embed): Embed()
  (hook_embed): HookPoint()
  (pos_embed): PosEmbed()
  (hook_pos_embed): HookPoint()
  (blocks): ModuleList(
    (0-1): 2 x TransformerBlock(
      (ln1): Identity()
      (attn): Attention(
        (hook_k): HookPoint()
        (hook_q): HookPoint()
        (hook_v): HookPoint()
        (hook_z): HookPoint()
        (hook_attn_scores): HookPoint()
        (hook_pattern): HookPoint()
        (hook_result): HookPoint()
      )
      (hook_q_input): HookPoint()
      (hook_k_input): HookPoint()
      (hook_v_input): HookPoint()
      (hook_attn_out): HookPoint()
      (hook_mlp_in): HookPoint()
      (hook_mlp_out): HookPoint()
      (hook_resid_pre): HookPoint()
      (hook_resid_post): HookPoint()
    )
  )
  (unembed): Unembed()
)


In [12]:
filename = "mech-interp-practice/models/permute_lists_model.pt"
state_dict = torch.load(filename)
model.load_state_dict(state_dict, strict=True)

<All keys matched successfully>

# Task Description

The models inputs are of the form:


```
[BOS, a_0, a_1, a_2, MID, idx_0, idx_1, idx_2, END, ans_0, ans_1, ans_2]
```


For example:


```
[BOS, 44, 40, 15, MID, 2, 1, 0, END, 15, 40, 44]
```


* BOS (52), MID (51), and END (50) are special tokens at the same position in every sequence
* The first 3 tokens after BOS, [a_0, a_1, a_2], is the original list the model needs to permute. Each list token ranges between [0,49] inclusive.
* The 3 tokens after MID, [idx_0, idx_1, idx_2] are the permutation indices. These are always some shuffling of {0, 1, 2}.
* The last three tokens after END, [ans_0, ans_1, ans_2], is the permuted list after applying the perm indices. In general, ans_i = a_{idx_i}. In the concrete example above: ans_0 = a_{idx_0} = a_2 = 15.


Note that the list length is fixed to 3 for every example. The model is trained to predict the ans tokens [ans_0, ans_1, ans_2] at positions [END, ans_0, ans_1] respectively. The model is trained with a causal mask so it can't peek ahead.


Below I provide a data loader and some example tokens that you can use to start your investigation:

In [13]:
def make_data_generator(cfg, batch_size, seed=0):
    torch.manual_seed(seed)
    BOS_TOKEN = cfg.d_vocab-1
    MID_TOKEN = cfg.d_vocab-2
    END_TOKEN = cfg.d_vocab-3
    while True:
        seq = torch.randint(0, MAX_INT, (batch_size, LIST_LEN))
        perm = torch.randperm(LIST_LEN)
        ans = seq[:, perm]

        bos_tensor = einops.repeat(torch.tensor(BOS_TOKEN), " -> i 1", i=batch_size)
        mid_tensor = einops.repeat(torch.tensor(MID_TOKEN), " -> i 1", i=batch_size)
        end_tensor = einops.repeat(torch.tensor(END_TOKEN), " -> i 1", i=batch_size)

        x = torch.cat([bos_tensor, seq, mid_tensor, einops.repeat(perm, "seq -> batch seq", batch=batch_size), end_tensor, ans], dim=-1)
        yield x

batch_size = 256
data_loader = make_data_generator(cfg, batch_size, seed=42)

test_data = []
sub_batch_size = 4
for i in range(batch_size // sub_batch_size):
    test_data.append(next(make_data_generator(cfg, sub_batch_size, seed=i)))

test_data = torch.cat(test_data, dim=0).to(device)
print(test_data.shape)
print(test_data[:5])

torch.Size([256, 12])
tensor([[52, 44, 39, 33, 51,  0,  2,  1, 50, 44, 33, 39],
        [52, 10, 13, 29, 51,  0,  2,  1, 50, 10, 29, 13],
        [52, 27,  3, 47, 51,  0,  2,  1, 50, 27, 47,  3],
        [52, 33,  1, 16, 51,  0,  2,  1, 50, 33, 16,  1],
        [52, 45, 39, 24, 51,  1,  2,  0, 50, 39, 24, 45]], device='cuda:0')


The model was trained to minimize cross entropy loss. I provide the loss function used in training below.

In [14]:
def loss_fn(logits, tokens):
    logits=logits[:, -LIST_LEN-1:-1, :]
    logits = logits.to(torch.float64)
    labels = tokens[:, -LIST_LEN:]

    log_probs = logits.log_softmax(dim=-1)
    correct_log_probs = log_probs.gather(dim=-1, index=labels[..., None])[..., 0]
    return -correct_log_probs.mean()

Here's a helper function to convert tokens to a list of strings that you can use for plotly / CircuitsVis labels:

In [15]:
def tokens_to_plotly_labels(tokens, incl_pos=True):
    if tokens.ndim==2:
        tokens = tokens[0]
    if incl_pos:
        res = [f"{tok}_{i}" for i, tok in enumerate(tokens)]
    else:
        res = list(map(str, tokens.tolist()))
    res[0] = 'BOS'
    res[LIST_LEN+1] = 'MID'
    res[2*LIST_LEN+2]= 'END'
    return res

print("incl_pos=True:", tokens_to_plotly_labels(test_data))
print("incl_pos=False:", tokens_to_plotly_labels(test_data, incl_pos=False))

incl_pos=True: ['BOS', '44_1', '39_2', '33_3', 'MID', '0_5', '2_6', '1_7', 'END', '44_9', '33_10', '39_11']
incl_pos=False: ['BOS', '44', '39', '33', 'MID', '0', '2', '1', 'END', '44', '33', '39']
